Initial Imports

In [2]:
import kinematics as kin
import transforms as tr
from visualization import VizScene
import numpy as np
import time
np.set_printoptions(precision=4, suppress=True)

Part 1

In [45]:
dh = np.array([[-np.pi/2,2.7,-.7,np.pi/2],
               [np.pi/2,0,0,-np.pi/2],
               [0,3.64,-.7,np.pi/2],
               [0,0,0,-np.pi/2],
               [0,3.74,-.1,np.pi/2],
               [0,0,0,-np.pi/2],
               [0,2.3,0,0]])

jt_types = ['r', 'r', 'r', 'r', 'r', 'r', 'r']

q_set_0 = [0, 0, 0, 0, 0, 0, 0]
q_set_1 = [-0.1135, -0.06481, 0.05024, 0.05561, 0.79077, -1.068, 2.979]
q_set_2 = [0.4836, -1.118, -0.3869, 2.2891, 1.8005, -1.5704, 3.0357]
q_set_3 = [0.4974, -1.1965, -0.8786, -0.0495, 3.05, -1.5455, 3.03575]
q_set_4 = [0.03106, 0.1020, -0.35742, 1.2049, 3.0576, -0.0360, 3.03613]

t_torso = np.array([[0.7071, 0.7071, 0, 0.06353],
                    [-0.7071, 0.7071, 0, -0.2597],
                    [0, 0, 1, 0.119],
                    [0, 0, 0, 1]])
arm = kin.SerialArm(dh, jt=jt_types, base=t_torso)

viz = VizScene()
viz.add_arm(arm, draw_frames=True)

viz.update(qs=[q_set_4])
viz.hold()